## 重投影

In [1]:
from osgeo import gdal, osr

In [2]:
path_img = "E:\\YNU\\5\\OpenSourceGIS\\Act_5\\OnClass\\chenggong_utm48.tif"
path_reproj = "E:\\YNU\\5\\OpenSourceGIS\\Act_5\\OnClass\\chenggong_wgs84_reproj.tif"

In [6]:
dset = gdal.Open(path_img) # 打开影像文件
proj = dset.GetProjection() # 获取投影信息
print(proj) # 输出投影信息

PROJCS["WGS 84 / UTM zone 48N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",105],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32648"]]


In [8]:
utm_epsg = 32648  # UTM坐标系
wgs84_epsg = 4326  # WGS84坐标系
utm = osr.SpatialReference()  # 创建空间参考对象
utm.ImportFromEPSG(utm_epsg)  # 导入UTM坐标系
wgs84 = osr.SpatialReference()  # 创建空间参考对象
wgs84.ImportFromEPSG(wgs84_epsg)  # 导入WGS84坐标系

0

In [18]:
wgs84.SetAxisMappingStrategy(osr.OAMS_TRADITIONAL_GIS_ORDER)  # 设置坐标轴映射策略(先经度，后纬度)
transform = osr.CoordinateTransformation(utm, wgs84)  # 定义坐标转换对象

geo_trans = dset.GetGeoTransform()  # 获取地理转换参数
print(geo_trans[0], geo_trans[3])  # 获取左上角角点坐标
print(geo_trans[1], geo_trans[5])  # 获取像素大小
print(geo_trans[2], geo_trans[4])  # 获取旋转角度

x_min_wgs84, y_min_wgs84, z = transform.TransformPoint(geo_trans[0], geo_trans[3], 0)  # 转换左上角角点坐标
x_max_wgs84, y_max_wgs84, z = transform.TransformPoint(geo_trans[0] + geo_trans[1] * dset.RasterXSize, geo_trans[3] + geo_trans[5] * dset.RasterYSize, 0)  # 转换右下角角点坐标
print(x_min_wgs84, y_min_wgs84)  # 输出左上角角点坐标
print(x_max_wgs84, y_max_wgs84)  # 输出右下角角点坐标

268167.02525260474 2765453.1872716597
10.421454790528236 -9.543567381150961
0.0 0.0
102.70321342725788 24.98683202712329
103.004567934636 24.700057015593973


In [24]:
x_max_utm = geo_trans[0] + geo_trans[1] * dset.RasterXSize ## utm坐标系右下角x坐标
y_max_utm = geo_trans[3] + geo_trans[5] * dset.RasterYSize ## utm坐标系右下角y坐标
print('utm坐标系右下角坐标为', x_max_utm, y_max_utm) ## 输出utm坐标系右下角坐标
x_max_wgs84, y_max_wgs84, _ = transform.TransformPoint(x_max_utm, y_max_utm, 0) ## 转换为wgs84坐标
print('wgs84坐标系右下角坐标为', x_max_wgs84, y_max_wgs84) ## 输出wgs84坐标系右下角坐标
x_res = (x_max_wgs84 - x_min_wgs84) / dset.RasterXSize ## 计算wgs84坐标系x方向分辨率
y_res = (y_max_wgs84 - y_min_wgs84) / dset.RasterYSize ## 计算wgs84坐标系y方向分辨率
print('wgs84坐标系分辨率为', x_res, y_res) ## 输出wgs84坐标系分辨率

utm坐标系右下角坐标为 298128.7077753734 2733205.4730907506
wgs84坐标系右下角坐标为 103.004567934636 24.700057015593973
wgs84坐标系分辨率为 0.00010481895908804231 -8.486978737180184e-05


In [20]:
geotrans_wgs84 = [102.70321342725788, 0.00010481895908804231, 0, 24.98683202712329, 0, -8.486978737180184e-05]  # WGS84坐标系的投影坐标转换参数

In [21]:
proj_utm_wkt = utm.ExportToWkt() # 获取投影的WKT字符串
proj_wgs84_wkt = wgs84.ExportToWkt() # 获取WGS84的WKT字符串
print(proj_utm_wkt) # 输出投影的WKT字符串

PROJCS["WGS 84 / UTM zone 48N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",105],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32648"]]


In [22]:
driver = gdal.GetDriverByName('GTiff') ## 创建数据驱动
dset_wgs84 = driver.Create(path_reproj, ## 输出文件的路径
                           xsize = dset.RasterXSize, ## 输出的大小等于原图的大小
                           ysize = dset.RasterYSize, ## 输出的大小等于原图的大小
                           bands = dset.RasterCount, ## 输出的波段数等于原图的波段数
                           eType = gdal.GDT_Int16, ## 输出数据的类型
                           )
# 相关信息的写入
dset_wgs84.SetGeoTransform(geotrans_wgs84) ## 写入原图的地理转换参数
dset_wgs84.SetProjection(proj_wgs84_wkt) ## 写入原图的投影信息

reproj = gdal.ReprojectImage(src_ds = dset, ## 原图
                             dst_ds = dset_wgs84, ## 输出图
                                src_wkt = proj_utm_wkt, ## 原图的投影信息
                                dst_wkt = proj_wgs84_wkt, ## 输出图的投影信息
                                eResampleAlg = gdal.GRA_Bilinear
                                )
dset_wgs84 = None ## 关闭输出图
